In [332]:
color2num = dict(
    gray=30,
    red=31,
    green=32,
    yellow=33,
    blue=34,
    magenta=35,
    cyan=36,
    white=37,
    crimson=38,
)

def colorize(string, color, bold=False, highlight=False):
    """
    Colorize a string.

    This function was originally written by John Schulman.
    """
    attr = []
    num = color2num[color]
    if highlight:
        num += 10
    attr.append(str(num))
    if bold:
        attr.append("1")
    return "\x1b[%sm%s\x1b[0m" % (";".join(attr), string)


def chunk(seq, size=32, start_id=0):
    chunked_seq = ""
    for j in range(start_id, size, 2):
        chunked_seq += seq[j-start_id:j-start_id+2] + " "
    chunked_seq = chunked_seq.strip(" ")
    chunked_seq += "\n"
    for i in range(size, len(seq), size):
        for j in range(i, i+size, 2):
            chunked_seq += seq[j-start_id:j-start_id+2] + " "
        chunked_seq = chunked_seq.strip(" ")
        chunked_seq += "\n"
    chunked_seq = chunked_seq.strip("\n")
    return chunked_seq

# Create random string

In [234]:
import random

random.seed(0)
num_plaintext_bytes = 64
block_bytes = 128 // 8

plaintext_origin = "".join(str(i*2)*block_bytes for i in range(num_plaintext_bytes//block_bytes)).encode()
random_plaintext = bytes(random.randrange(256) for _ in range(num_plaintext_bytes))
with open("h1_plaintext_origin", "wb") as f:
    f.write(plaintext_origin)

print(plaintext_origin)
print(len(random_plaintext))

b'0000000000000000222222222222222244444444444444446666666666666666'
64


# Encryption

available cipher:

```text
aes-[128|192|256]-cbc  128/192/256 bit AES in CBC mode
aes[128|192|256]       Alias for aes-[128|192|256]-cbc
aes-[128|192|256]-cfb  128/192/256 bit AES in 128 bit CFB mode
aes-[128|192|256]-cfb1 128/192/256 bit AES in 1 bit CFB mode
aes-[128|192|256]-cfb8 128/192/256 bit AES in 8 bit CFB mode
aes-[128|192|256]-ctr  128/192/256 bit AES in CTR mode
aes-[128|192|256]-ecb  128/192/256 bit AES in ECB mode
aes-[128|192|256]-ofb  128/192/256 bit AES in OFB mode
```

In [235]:
random.seed(0)

Key = bytes(random.randrange(256) for _ in range(block_bytes)).hex()
IV = bytes(random.randrange(256) for _ in range(block_bytes)).hex()
print(block_bytes, Key, IV)

16 c5d71484f8cf9bf4b76f47904730804b 9e3225a9f133b5dea168f4e2851f072f


## ecb

### cipher

In [236]:
!openssl enc -e -aes-128-ecb -in h1_plaintext_origin -out h1_ciphertext_ecb -K c5d71484f8cf9bf4b76f47904730804b

### corrupt ciphertext

In [237]:
with open("h1_ciphertext_ecb", "rb") as f:
    ciphertext_ecb = f.read()
ciphertext_ecb_corrupted = bytearray(ciphertext_ecb)
ciphertext_ecb_corrupted[27] += 1
ciphertext_ecb_corrupted = bytes(ciphertext_ecb_corrupted)
with open("h1_ciphertext_ecb_corrupted", "wb") as f:
    f.write(ciphertext_ecb_corrupted)

### decipher

In [277]:
!openssl enc -d -aes-128-ecb -in h1_ciphertext_ecb -out h1_plaintext_deciphered_ecb -K c5d71484f8cf9bf4b76f47904730804b
!openssl enc -d -aes-128-ecb -in h1_ciphertext_ecb_corrupted -out h1_plaintext_deciphered_ecb_corrupted -K c5d71484f8cf9bf4b76f47904730804b

### compare

In [337]:
with open("h1_plaintext_deciphered_ecb", "rb") as f:
    plaintext_deciphered_ecb = f.read()
with open("h1_plaintext_deciphered_ecb_corrupted", "rb") as f:
    plaintext_deciphered_ecb_corrupted = f.read()

print(colorize("cipher: aes-128-ecb", "magenta", True))
print("original plaintext:\n", colorize(chunk(plaintext_origin.hex()), "cyan"))
print("ciphertext:\n", chunk(ciphertext_ecb.hex()))
print("corrupted ciphertext:\n", chunk(ciphertext_ecb_corrupted[:27].hex()) 
    + " " + colorize(chunk(ciphertext_ecb_corrupted[27:28].hex()), "red") 
    + " " + chunk(ciphertext_ecb_corrupted[28:].hex(), start_id=12*2))
print("deciphered plaintext from ciphertext:\n", colorize(chunk(plaintext_deciphered_ecb.hex()), "blue"))
print("deciphered plaintext from corrupted ciphertext:\n", colorize(chunk(plaintext_deciphered_ecb_corrupted.hex()), "yellow"))

cipher: aes-128-ecb
original plaintext:
 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32
34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34
36 36 36 36 36 36 36 36 36 36 36 36 36 36 36 36
ciphertext:
 eb c6 af 7b ae 0b f0 66 4c 6b 5f 71 91 89 29 30
32 dc e0 02 b1 78 18 82 c6 6b dd 38 b1 7d fc 72
36 a8 1f a3 d4 8f 4c 0c 1a 15 8e 82 46 59 df 81
f3 a1 ca 4a 62 9c 09 ec b4 14 0f f9 13 c1 cc 9e
cb 01 a9 97 5e 27 39 13 0c 3d b1 e5 47 bc bf e0
corrupted ciphertext:
 eb c6 af 7b ae 0b f0 66 4c 6b 5f 71 91 89 29 30
32 dc e0 02 b1 78 18 82 c6 6b dd 39 b1 7d fc 72
36 a8 1f a3 d4 8f 4c 0c 1a 15 8e 82 46 59 df 81
f3 a1 ca 4a 62 9c 09 ec b4 14 0f f9 13 c1 cc 9e
cb 01 a9 97 5e 27 39 13 0c 3d b1 e5 47 bc bf e0
deciphered plaintext from ciphertext:
 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32
34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34
36 36 36 36 36 36 36 36 36 36 36 36 36 36 36 36
deciphered plaintext

## cbc

In [338]:
### cipher
!openssl enc -e -aes-128-cbc -in h1_plaintext_origin -out h1_ciphertext_cbc -K c5d71484f8cf9bf4b76f47904730804b -iv 9e3225a9f133b5dea168f4e2851f072f

### corrupt ciphertext
with open("h1_ciphertext_cbc", "rb") as f:
    ciphertext_cbc = f.read()
ciphertext_cbc_corrupted = bytearray(ciphertext_cbc)
ciphertext_cbc_corrupted[27] += 1
ciphertext_cbc_corrupted = bytes(ciphertext_cbc_corrupted)
with open("h1_ciphertext_cbc_corrupted", "wb") as f:
    f.write(ciphertext_cbc_corrupted)

### decipher
!openssl enc -d -aes-128-cbc -in h1_ciphertext_cbc -out h1_plaintext_deciphered_cbc -K c5d71484f8cf9bf4b76f47904730804b -iv 9e3225a9f133b5dea168f4e2851f072f
!openssl enc -d -aes-128-cbc -in h1_ciphertext_cbc_corrupted -out h1_plaintext_deciphered_cbc_corrupted -K c5d71484f8cf9bf4b76f47904730804b -iv 9e3225a9f133b5dea168f4e2851f072f

### compare
with open("h1_plaintext_deciphered_cbc", "rb") as f:
    plaintext_deciphered_cbc = f.read()
with open("h1_plaintext_deciphered_cbc_corrupted", "rb") as f:
    plaintext_deciphered_cbc_corrupted = f.read()

print(colorize("cipher: aes-128-cbc", "magenta", True))
print("original plaintext:\n", colorize(chunk(plaintext_origin.hex()), "cyan"))
print("ciphertext:\n", chunk(ciphertext_cbc.hex()))
print("corrupted ciphertext:\n", chunk(ciphertext_cbc_corrupted[:27].hex()) 
    + " " + colorize(chunk(ciphertext_cbc_corrupted[27:28].hex()), "red") 
    + " " + chunk(ciphertext_cbc_corrupted[28:].hex(), start_id=12*2))
print("deciphered plaintext from ciphertext:\n", colorize(chunk(plaintext_deciphered_cbc.hex()), "blue"))
print("deciphered plaintext from corrupted ciphertext:\n", colorize(chunk(plaintext_deciphered_cbc_corrupted.hex()), "yellow"))

cipher: aes-128-cbc
original plaintext:
 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32
34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34
36 36 36 36 36 36 36 36 36 36 36 36 36 36 36 36
ciphertext:
 d4 f3 8a d9 16 ed 54 d0 c1 71 7a 86 3b 46 bb 34
8d de 79 61 d0 e1 63 fe 03 31 6a f7 25 3c b1 2a
d4 53 06 8b 1c ca 7c 52 b5 2d 46 1d 8e 14 20 ad
42 a2 7f d7 3a 81 42 85 f9 81 4a 17 38 a4 26 7e
c0 2e db 7d 49 9a 0e 15 be 04 40 2c 70 0a 91 16
corrupted ciphertext:
 d4 f3 8a d9 16 ed 54 d0 c1 71 7a 86 3b 46 bb 34
8d de 79 61 d0 e1 63 fe 03 31 6a f8 25 3c b1 2a
d4 53 06 8b 1c ca 7c 52 b5 2d 46 1d 8e 14 20 ad
42 a2 7f d7 3a 81 42 85 f9 81 4a 17 38 a4 26 7e
c0 2e db 7d 49 9a 0e 15 be 04 40 2c 70 0a 91 16
deciphered plaintext from ciphertext:
 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32
34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34
36 36 36 36 36 36 36 36 36 36 36 36 36 36 36 36
deciphered plaintext

### Conclusion

A one-bit change to the ciphertext causes complete corruption of the corresponding block of plaintext, and inverts the corresponding bit in the following block of plaintext, but the rest of the blocks remain intact. 

## ofb

In [340]:
### cipher
!openssl enc -e -aes-128-ofb -in h1_plaintext_origin -out h1_ciphertext_ofb -K c5d71484f8cf9bf4b76f47904730804b -iv 9e3225a9f133b5dea168f4e2851f072f

### corrupt ciphertext
with open("h1_ciphertext_ofb", "rb") as f:
    ciphertext_ofb = f.read()
ciphertext_ofb_corrupted = bytearray(ciphertext_ofb)
ciphertext_ofb_corrupted[27] += 1
ciphertext_ofb_corrupted = bytes(ciphertext_ofb_corrupted)
with open("h1_ciphertext_ofb_corrupted", "wb") as f:
    f.write(ciphertext_ofb_corrupted)

### decipher
!openssl enc -d -aes-128-ofb -in h1_ciphertext_ofb -out h1_plaintext_deciphered_ofb -K c5d71484f8cf9bf4b76f47904730804b -iv 9e3225a9f133b5dea168f4e2851f072f
!openssl enc -d -aes-128-ofb -in h1_ciphertext_ofb_corrupted -out h1_plaintext_deciphered_ofb_corrupted -K c5d71484f8cf9bf4b76f47904730804b -iv 9e3225a9f133b5dea168f4e2851f072f

### compare
with open("h1_plaintext_deciphered_ofb", "rb") as f:
    plaintext_deciphered_ofb = f.read()
with open("h1_plaintext_deciphered_ofb_corrupted", "rb") as f:
    plaintext_deciphered_ofb_corrupted = f.read()

print(colorize("cipher: aes-128-ofb", "magenta", True))
print("original plaintext:\n", colorize(chunk(plaintext_origin.hex()), "cyan"))
print("ciphertext:\n", chunk(ciphertext_ofb.hex()))
print("corrupted ciphertext:\n", chunk(ciphertext_ofb_corrupted[:27].hex()) 
    + " " + colorize(chunk(ciphertext_ofb_corrupted[27:28].hex()), "red") 
    + " " + chunk(ciphertext_ofb_corrupted[28:].hex(), start_id=12*2))
print("deciphered plaintext from ciphertext:\n", colorize(chunk(plaintext_deciphered_ofb.hex()), "blue"))
print("deciphered plaintext from corrupted ciphertext:\n", colorize(chunk(plaintext_deciphered_ofb_corrupted.hex()), "yellow"))

cipher: aes-128-ofb
original plaintext: 
 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32
34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34
36 36 36 36 36 36 36 36 36 36 36 36 36 36 36 36
ciphertext:
 b2 b8 3a 8c f8 76 3c d3 16 3b 16 59 50 11 5f 38
78 bc 07 49 00 f7 eb f5 ef 8c a2 82 bf 5d 10 1b
c7 16 1b 9c 8c 87 e4 1c ed ab fb cf c8 2e 7e 57
af b6 d1 c4 4c 4a 81 85 b7 69 d7 c1 3d 83 db ba
corrupted ciphertext:
 b2 b8 3a 8c f8 76 3c d3 16 3b 16 59 50 11 5f 38
78 bc 07 49 00 f7 eb f5 ef 8c a2 83 bf 5d 10 1b
c7 16 1b 9c 8c 87 e4 1c ed ab fb cf c8 2e 7e 57
af b6 d1 c4 4c 4a 81 85 b7 69 d7 c1 3d 83 db ba
deciphered plaintext from ciphertext:
 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32
34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34
36 36 36 36 36 36 36 36 36 36 36 36 36 36 36 36
deciphered plaintext from corrupted ciphertext:
 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
32 32 32 32 32 32 

### cfb

In [341]:
### cipher
!openssl enc -e -aes-128-cfb -in h1_plaintext_origin -out h1_ciphertext_cfb -K c5d71484f8cf9bf4b76f47904730804b -iv 9e3225a9f133b5dea168f4e2851f072f

### corrupt ciphertext
with open("h1_ciphertext_cfb", "rb") as f:
    ciphertext_cfb = f.read()
ciphertext_cfb_corrupted = bytearray(ciphertext_cfb)
ciphertext_cfb_corrupted[27] += 1
ciphertext_cfb_corrupted = bytes(ciphertext_cfb_corrupted)
with open("h1_ciphertext_cfb_corrupted", "wb") as f:
    f.write(ciphertext_cfb_corrupted)

### decipher
!openssl enc -d -aes-128-cfb -in h1_ciphertext_cfb -out h1_plaintext_deciphered_cfb -K c5d71484f8cf9bf4b76f47904730804b -iv 9e3225a9f133b5dea168f4e2851f072f
!openssl enc -d -aes-128-cfb -in h1_ciphertext_cfb_corrupted -out h1_plaintext_deciphered_cfb_corrupted -K c5d71484f8cf9bf4b76f47904730804b -iv 9e3225a9f133b5dea168f4e2851f072f

### compare
with open("h1_plaintext_deciphered_cfb", "rb") as f:
    plaintext_deciphered_cfb = f.read()
with open("h1_plaintext_deciphered_cfb_corrupted", "rb") as f:
    plaintext_deciphered_cfb_corrupted = f.read()

print(colorize("cipher: aes-128-cfb", "magenta", True))
print("original plaintext:\n", colorize(chunk(plaintext_origin.hex()), "cyan"))
print("ciphertext:\n", chunk(ciphertext_cfb.hex()))
print("corrupted ciphertext:\n", chunk(ciphertext_cfb_corrupted[:27].hex()) 
    + " " + colorize(chunk(ciphertext_cfb_corrupted[27:28].hex()), "red") 
    + " " + chunk(ciphertext_cfb_corrupted[28:].hex(), start_id=12*2))
print("deciphered plaintext from ciphertext:\n", colorize(chunk(plaintext_deciphered_cfb.hex()), "blue"))
print("deciphered plaintext from corrupted ciphertext:\n", colorize(chunk(plaintext_deciphered_cfb_corrupted.hex()), "yellow"))

cipher: aes-128-cfb
original plaintext:
 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32
34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34
36 36 36 36 36 36 36 36 36 36 36 36 36 36 36 36
ciphertext:
 b2 b8 3a 8c f8 76 3c d3 16 3b 16 59 50 11 5f 38
ce 1b 78 7c 82 6b ce 53 1a e2 82 2b 17 e3 82 94
9b dc 40 94 09 c2 65 15 80 59 3a c9 48 5f 8c c5
21 90 a6 e1 b0 50 a7 7e 6c 4b a3 b7 10 7b 72 00
corrupted ciphertext:
 b2 b8 3a 8c f8 76 3c d3 16 3b 16 59 50 11 5f 38
ce 1b 78 7c 82 6b ce 53 1a e2 82 2c 17 e3 82 94
9b dc 40 94 09 c2 65 15 80 59 3a c9 48 5f 8c c5
21 90 a6 e1 b0 50 a7 7e 6c 4b a3 b7 10 7b 72 00
deciphered plaintext from ciphertext:
 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32
34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34
36 36 36 36 36 36 36 36 36 36 36 36 36 36 36 36
deciphered plaintext from corrupted ciphertext:
 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
32 32 32 32 32 32 3

### ctr

In [343]:
### cipher
!openssl enc -e -aes-128-ctr -in h1_plaintext_origin -out h1_ciphertext_ctr -K c5d71484f8cf9bf4b76f47904730804b -iv 9e3225a9f133b5dea168f4e2851f072f

### corrupt ciphertext
with open("h1_ciphertext_ctr", "rb") as f:
    ciphertext_ctr = f.read()
ciphertext_ctr_corrupted = bytearray(ciphertext_ctr)
ciphertext_ctr_corrupted[27] += 1
ciphertext_ctr_corrupted = bytes(ciphertext_ctr_corrupted)
with open("h1_ciphertext_ctr_corrupted", "wb") as f:
    f.write(ciphertext_ctr_corrupted)

### decipher
!openssl enc -d -aes-128-ctr -in h1_ciphertext_ctr -out h1_plaintext_deciphered_ctr -K c5d71484f8cf9bf4b76f47904730804b -iv 9e3225a9f133b5dea168f4e2851f072f
!openssl enc -d -aes-128-ctr -in h1_ciphertext_ctr_corrupted -out h1_plaintext_deciphered_ctr_corrupted -K c5d71484f8cf9bf4b76f47904730804b -iv 9e3225a9f133b5dea168f4e2851f072f

### compare
with open("h1_plaintext_deciphered_ctr", "rb") as f:
    plaintext_deciphered_ctr = f.read()
with open("h1_plaintext_deciphered_ctr_corrupted", "rb") as f:
    plaintext_deciphered_ctr_corrupted = f.read()

print(colorize("cipher: aes-128-ctr", "magenta", True))
print("original plaintext:\n", colorize(chunk(plaintext_origin.hex()), "cyan"))
print("ciphertext:\n", chunk(ciphertext_ctr.hex()))
print("corrupted ciphertext:\n", chunk(ciphertext_ctr_corrupted[:27].hex()) 
    + " " + colorize(chunk(ciphertext_ctr_corrupted[27:28].hex()), "red") 
    + " " + chunk(ciphertext_ctr_corrupted[28:].hex(), start_id=12*2))
print("deciphered plaintext from ciphertext:\n", colorize(chunk(plaintext_deciphered_ctr.hex()), "blue"))
print("deciphered plaintext from corrupted ciphertext:\n", colorize(chunk(plaintext_deciphered_ctr_corrupted.hex()), "yellow"))

cipher: aes-128-ctr
original plaintext:
 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32
34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34
36 36 36 36 36 36 36 36 36 36 36 36 36 36 36 36
ciphertext:
 b2 b8 3a 8c f8 76 3c d3 16 3b 16 59 50 11 5f 38
51 5e 8f 63 18 e7 63 0a a6 84 64 78 bc c5 dc 3b
89 32 99 a1 a1 61 be 1a ee ef 82 61 3e 3a 72 d8
01 01 e4 71 6e c8 19 c1 ba 83 60 40 5b 58 19 b7
corrupted ciphertext:
 b2 b8 3a 8c f8 76 3c d3 16 3b 16 59 50 11 5f 38
51 5e 8f 63 18 e7 63 0a a6 84 64 79 bc c5 dc 3b
89 32 99 a1 a1 61 be 1a ee ef 82 61 3e 3a 72 d8
01 01 e4 71 6e c8 19 c1 ba 83 60 40 5b 58 19 b7
deciphered plaintext from ciphertext:
 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32
34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34
36 36 36 36 36 36 36 36 36 36 36 36 36 36 36 36
deciphered plaintext from corrupted ciphertext:
 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30 30
32 32 32 32 32 32 3